# Challenge Description

Analyze user engagement and retention with respect to TripAdvisor links.
Are there particular features or pages that lead to higher engagement? 
Identify and visualize drop-off points or
areas where users might abandon the journey to TripAdvisor.

# Imports

In [48]:
import os
import pandas as pd
from collections import Counter

import plotly
import tldextract

# Functions

In [58]:
def get_retention(data: pd.DataFrame, 
                  n_file: int=0) -> pd.DataFrame:
    '''Returns a dataframe where to see with links of tripadvisor
    increments the engagement'''

    # start variables id and current journey
    engagement = []
    links = []
    
    # check every row
    data['targeturl'] = data['targeturl'].apply(get_clean_link)
    
    for idx in range(len(data.index) - 1):
        if data.domain.iloc[idx] == 'tripadvisor':
            if data.domain.iloc[idx + 1] == 'tripadvisor':
                engagement.append(True)
                links.append(data.targeturl.iloc[idx])
            else:
                engagement.append(False)
                links.append(data.targeturl.iloc[idx])
    
    # drop empty rows and save the file
    retention = pd.DataFrame()
    retention['link'] = links
    retention['engagement'] = engagement 
    retention = retention.dropna(how='all')
    retention.to_parquet(f'data/retention.parquet')
    
    return retention

In [33]:
def get_clean_data(file: str) -> pd.DataFrame:
    '''Returns the data cleaned 
       It also saves the new file for future iteration'''
    if os.path.exists(f'data/clean_{file}'):
        return pd.read_parquet(f'data/clean_{file}')

In [68]:
def get_clean_link(url: str) -> str:
    for word in ['https//', 'www.', 'tripadvisor', '.com', '.co.uk', '/', 'https:', '#']:
        url = url.replace(word, '')
    return url

# Data Import

In [63]:
# prepare and import all data.
retention = pd.DataFrame()
for n in range(48):
    file = f'data_{n}.parquet'
    data = get_clean_data(file)
    new_retention = get_retention(data)
    retention = pd.concat([retention, new_retention], axis=1)

# Split Data

In [64]:
higher_engagement = retention[retention['engagement'] == True]
higher_engagement

,link,engagement
3,.well-knownchange-password,True
4,settings-cp,True
5,forgotpassword-e__2f__settings__2d__cp,True
6,memberresetpassword?resetflowemail=REMOVED&tt=...,True
7,passwordreset-a_prq.g1f342387a3bd64a16982cd93d...,True
...,...,...
4172,AttractionProductReview-g150800-d11455238-Colo...,True
4173,ShoppingCartConfirmation?reservationId=C8Ta-GP...,True
4183,Restaurants-g32640-Lodi_California.html,True
4184,Restaurant_Review-g32640-d4498989-Reviews-The_...,True


In [75]:
pd.Series(higher_engagement_patterns).to_csv('data/higher_engagement.csv')
pd.Series(drop_off_patterns).to_csv('data/drop_off.csv')

In [65]:
drop_off = retention[retention['engagement'] == False]
drop_off

,link,engagement
0,restaurant_review-g12909146-d4444636-reviews-t...,False
1,hotel_review-g44030-d295308-reviews-hotel_ches...,False
2,hotel_review-g44030-d295308-reviews-hotel_ches...,False
9,passwordreset,False
10,hotels-g61000-yosemite_national_park_californi...,False
...,...,...
4203,Attraction_Products-g32482-Healdsburg_Sonoma_C...,False
4204,ShowTopic-g187463-i183-k10542910-Do_I_need_a_c...,False
4205,ShowTopic-g187463-i183-k858544-Do_I_need_a_car...,False
4206,ShowTopic-g29221-i1825-k4398088-Is_moving_to_M...,False


In [70]:
higher_engagement_patterns = Counter(higher_engagement.link).most_common(25)

In [71]:
drop_off_patterns = Counter(drop_off.link).most_common(25)

# Conclusion

tripadvisor loses engagements increases and drops off deppending on the review and comments from other people in the atraction/place they are seeing, a good review increase the engagement while a bad review tends to lead to a drop off